# Intermediate Machine Learning
(https://www.kaggle.com/learn/intermediate-machine-learning)

# --- 1. Missing Values ---

## Three Approaches
- Drop Columns with Missing Values
- Imputation (for instance, fil in the mean value)
- Extension to Imputation (impute missing values and add columns that show the location of imputed entries)

Working with the Melbourne Housing Dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Load the data
data = pd.read_csv('./melb_data.csv')

# Select target
y = data.Price

# To keep simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

Score_datasets 

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [4]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Score  from 1st Approach (Drop Columns with Missing Values)

In [7]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
print(cols_with_missing)

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values): ")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

['Car', 'BuildingArea', 'YearBuilt']
MAE from Approach 1 (Drop columns with missing values): 
183550.22137772635


## Score  from 2nd Approach (Imputation)

In [11]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


## Score  from 3rd Approach (Extenstion to Imputation)

In [14]:
# Make copy to avoid changing original data (when imputing)

X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extenstion to Imputation) :")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extenstion to Imputation) :
178927.503183954


Conclusion\
As is common, imputing missing values (in Approach 2 and Approach 3) yielded better results, relative to when we simply dropped columns with missing values (in Approach 1).

In [21]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


# --- 2. Categorical Variables ---

A **categorical variable** takes only a limited number of values.

## Three Approaches
- Drop Categorical Variables (work well if the columnds did not contain useful information)
- Ordinal Encoding (It assigns each unique value a different integer [ordinal variables] )
- One-Hot Encoding (It creates new columns indicating the presence of each possible value)


### Ordinal Encoding
# <img src="tEogUAr.png" alt="Orindal Encoding" width = "1000" heigth = "1000">

### One-Hot Encoding
# <img src="TW5m0aJ.png" alt="One-Hot Encoding" width = "1000" heigth = "1000">